In [ ]:
import sys
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:,.2f}'.format)
import altair as alt
import matplotlib.pyplot as plt


In [ ]:
df67 = pd.read_excel('67.xlsx')
df67['AMOUNT'] = df67['AMOUNT'].astype(float)
df67.info()

In [ ]:
df67.loc[df67['FISCAL_YEAR'] == 2024, 'AMOUNT'].sum()



In [ ]:
df67 = df67.drop(df67[df67['FISCAL_YEAR'] != 2024].index)
df67['AMOUNT'].sum()

In [ ]:
df67['MINISTRY'].unique()

In [ ]:
df68 = pd.read_excel('68.xlsx')
df68['AMOUNT'] = df68['AMOUNT'].astype(float)
df68.loc[df68['FISCAL_YEAR'] == 2025, 'AMOUNT'].sum()


In [ ]:
df68 = df68.drop(df68[df68['FISCAL_YEAR'] != 2025].index)
df68['AMOUNT'].sum()

In [ ]:
df66 = pd.read_excel('66.xlsx')
df66.loc[df66['FISCAL_YEAR'] == 2023, 'AMOUNT'].sum()

In [ ]:
df66 = df66.drop(df66[df66['FISCAL_YEAR'] != 2023].index)
df66['AMOUNT'].sum()

In [ ]:
df67.info()

In [ ]:
liscol = ['FISCAL_YEAR','MINISTRY','BUDGETARY_UNIT','BUDGET_PLAN','OUTPUT','PROJECT','ITEM_DESCRIPTION','AMOUNT']

In [ ]:
df66_clean = df66[liscol]
df67_clean = df67[liscol]
df68_clean = df68[liscol]
df66_67_68 = pd.concat([df66_clean,df67_clean,df68_clean]).reset_index(drop=True)
df66_67_68.info()
df66_67_68['MINISTRY'] = df66_67_68['MINISTRY'].replace('ส่วนราชการไม่สังกัดสำนักนายกรัฐมนตรี กระทรวง หรือทบวงและหน่วยงานภายใต้การควบคุมดูแลของนายกรัฐมนตรี','ส่วนราชการไม่สังกัดสำนักนายกรัฐมนตรี กระทรวง หรือทบวง และหน่วยงานภายใต้การควบคุมดูแลของนายกรัฐมนตรี')
df66_67_68['MINISTRY'] = df66_67_68['MINISTRY'].replace('ทุนหมุนเวียนที่มีฐานะเป็นนิติบุคคล','ทุนหมุนเวียน')
df66_67_68['MINISTRY'] = df66_67_68['MINISTRY'].replace('ทุนหมุนเวียนที่ไม่มีฐานะเป็นนิติบุคคล','ทุนหมุนเวียน')
df66_67_68['MINISTRY'] = df66_67_68['MINISTRY'].replace('ส่วนราชการไม่สังกัดสำนักนายกรัฐมนตรีฯ','ส่วนราชการไม่สังกัดสำนักนายกรัฐมนตรี กระทรวง หรือทบวง และหน่วยงานภายใต้การควบคุมดูแลของนายกรัฐมนตรี')

In [ ]:
df66_67_68.tail()

In [ ]:
df66_67_68['AMOUNT'].sum()

In [ ]:
group = df66_67_68[['MINISTRY','AMOUNT']].groupby(['MINISTRY']).sum().reset_index()
group = group.sort_values(by='AMOUNT',ascending=False).reset_index(drop=True)
bar_chart = df66_67_68[['AMOUNT','FISCAL_YEAR']].groupby(['FISCAL_YEAR']).sum().reset_index()

In [ ]:
#barchart งบแต่ละกระทรวง+เส้นค่าเฉลี่ย
A = alt.Chart(group).mark_bar().encode(
    x=alt.X('MINISTRY', sort='-y'),  # เรียงลำดับจากมากไปน้อยตามค่า y (AMOUNT)
    y='AMOUNT',
    color='MINISTRY',
    tooltip=[
        alt.Tooltip('MINISTRY', title='Ministry'),
        alt.Tooltip('AMOUNT', title='Amount', format=",.2f")  # ฟอร์แมตตัวเลข , และไม่มีฐาน E
    ]
).interactive()
mean_line = alt.Chart(group).mark_rule(color='red').encode(
    y='mean(AMOUNT):Q',tooltip = alt.Tooltip('mean(AMOUNT):Q', title='Average Amount',format=",.2f"))   # ใช้ mean ของ AMOUNT เพื่อกำหนดตำแหน่ง Y)

A_meanline = A+mean_line

In [ ]:
barsum = alt.Chart(bar_chart).mark_bar().encode(
    x="AMOUNT",
    y="FISCAL_YEAR:O",
    tooltip=
        alt.Tooltip('AMOUNT', title='Amount', format=",.2f")  # ฟอร์แมตตัวเลข , และไม่มีฐาน E
)
barsum & A_meanline

In [ ]:
C = df66_67_68.loc[df66_67_68['MINISTRY'] == 'งบกลาง', 'BUDGETARY_UNIT'].unique()
C

In [ ]:
group3 = df66_67_68[['BUDGETARY_UNIT','AMOUNT']].groupby(['BUDGETARY_UNIT']).sum()
group3.sort_values(by='AMOUNT',ascending=False)

In [ ]:
dropdown = alt.binding_select(options=df66_67_68['MINISTRY'].unique().tolist(), name='Select Ministry:')
selector = alt.selection_single(fields=['MINISTRY'], bind=dropdown, init={'MINISTRY': 'กระทรวงกลาโหม'})
group2 = df66_67_68[['MINISTRY','AMOUNT','FISCAL_YEAR']].groupby(['FISCAL_YEAR','MINISTRY']).sum().reset_index()
B = alt.Chart(group2).mark_bar().encode(
    x=alt.X('FISCAL_YEAR:O', sort='-y'),  # แกน X เป็นปี (Ordinal)
    y='AMOUNT:Q',  # แกน Y เป็นจำนวนเงิน (Quantitative)
    color='MINISTRY',  # ใช้สีตาม MINISTRY
    tooltip=[
        alt.Tooltip('MINISTRY', title='Ministry'),
        alt.Tooltip('AMOUNT', title='Amount', format=",.2f"),  # ฟอร์แมตตัวเลข , และไม่มีฐาน E
    ]
).transform_filter(selector).properties(title='Bar Chart by Ministry')

# แสดงผล
B.add_selection(selector)

In [ ]:
group2 = df66_67_68[['MINISTRY','AMOUNT','FISCAL_YEAR']].groupby(['FISCAL_YEAR','MINISTRY']).sum().reset_index()
group2.info()
group2_1 = group2.pivot_table( index='MINISTRY', columns='FISCAL_YEAR', values='AMOUNT')
group2_1 = group2_1.reset_index()
group2_1['MINISTRY'].unique()

In [ ]:
group3 = group2
group3 = group3[group3['MINISTRY'] != 'รายจ่ายเพื่อชดใช้เงินคงคลัง']
group3['rank'] = group3.groupby('FISCAL_YEAR')['AMOUNT'].rank(ascending=False)


group3['rank'] = group3['rank'].astype(int)
group3_1 = group3.pivot_table( index='MINISTRY', columns='FISCAL_YEAR', values='rank')
group3_1


group3_1[2023] = group3_1[2023].astype(int)
group3_1[2024] = group3_1[2024].astype(int)
group3_1[2025] = group3_1[2025].astype(int)
group3_1['Mean_of_Absolute_change']=(abs(group3_1[2024]-group3_1[2023])+abs(group3_1[2025]-group3_1[2024]))/2
group3_1.sort_values(by=[2023,2024,2025])







In [ ]:
group2_1['2023_to_2024_Change'] = ((group2_1[2024]-group2_1[2023])/group2_1[2023])*100
group2_1['2024_to_2025_Change'] = ((group2_1[2025]-group2_1[2024])/group2_1[2024])*100
group2_1 = group2_1.dropna()
heat = group2_1.drop(columns=[2023,2024,2025])
heat = heat.sort_values(by=['2023_to_2024_Change','2024_to_2025_Change'])
heat = heat.reset_index(drop=True)
heat.index = range(1, len(heat) + 1)
heat_styled = heat.style.background_gradient(cmap='RdYlGn').format(precision=2)
heat_styled

In [ ]:
group2_2 = group2.pivot_table( index='MINISTRY', columns='FISCAL_YEAR', values='AMOUNT')
group2_2['2023_to_2024_Change'] = group2_2[2024]-group2_2[2023]
group2_2['2024_to_2025_Change'] = group2_2[2025]-group2_2[2024]
group2_2 = group2_2.drop(columns=[2023,2024,2025])
group2_2 = group2_2.dropna()
group2_2 = group2_2.sort_values(by=['2023_to_2024_Change','2024_to_2025_Change']).reset_index()
group2_2.index = range(1, len(group2_2) + 1)
group2_2_styled = group2_2.style \
    .background_gradient(cmap='RdYlGn') \
    .format({
        '2023_to_2024_Change': '{:,.2f}',  # จัดฟอร์แมตการเปลี่ยนแปลงปีงบประมาณ
        '2024_to_2025_Change': '{:,.2f}'   # จัดฟอร์แมตการเปลี่ยนแปลงปีงบประมาณ
    })





group2_2_styled

In [ ]:
map_df = df66_67_68.loc[
    (df66_67_68['MINISTRY'] == 'จังหวัดและกลุ่มจังหวัด') &
    (~df66_67_68['BUDGETARY_UNIT'].str.contains('กลุ่มจังหวัดภาค')),
    :
]
map_df['BUDGETARY_UNIT'] = map_df['BUDGETARY_UNIT'].replace('จังหวัดนครสวรรค','จังหวัดนครสวรรค์')
map_df['BUDGETARY_UNIT'] = map_df['BUDGETARY_UNIT'].replace('จังหวัดล่าปาง','จังหวัดลำปาง')
map_df['BUDGETARY_UNIT'] = map_df['BUDGETARY_UNIT'].replace('จังหวัดสุราษฎ์ธานี','จังหวัดสุราษฎร์ธานี')
map_df['BUDGETARY_UNIT'] = map_df['BUDGETARY_UNIT'].replace('จังหวัดหนองบัวล่าภู','จังหวัดหนองบัวลำภู')
map_df['BUDGETARY_UNIT'] = map_df['BUDGETARY_UNIT'].replace('จังหวัดล่าพูน','จังหวัดลำพูน')
map_df['BUDGETARY_UNIT'] = map_df['BUDGETARY_UNIT'].replace('จังหวัดย','จังหวัดหนองคาย')

map_df['BUDGETARY_UNIT'].unique()

In [ ]:
groupmap_df = map_df.groupby(['FISCAL_YEAR', 'BUDGETARY_UNIT']).sum('AMOUNT').reset_index()
groupmap_df['BUDGETARY_UNIT'] = groupmap_df['BUDGETARY_UNIT'].str.replace('จังหวัด', '', regex=False)
groupmap_df

In [ ]:
import json
import json

# โหลดไฟล์ GeoJSON ของจังหวัดในประเทศไทย
try:
    with open('thailand-provinces.geojson', 'r', encoding='utf-8') as f:
        provinces_geo = json.load(f)

    print("GeoJSON loaded successfully!")
    print("Number of features:", len(provinces_geo['features']))

    # ตรวจสอบข้อมูลแรกใน features
    if 'features' in provinces_geo:
        first_feature = provinces_geo['features'][0]
        print("First feature properties:", first_feature['properties'])
except json.JSONDecodeError as e:
    print("Error loading GeoJSON:", e)
except Exception as e:
    print("An error occurred:", e)

features = pd.json_normalize(provinces_geo['features'])
features['properties.NAME_1'].unique()
B1 = {'type':'Feature','properties.NAME_0':'Thailand','properties.NAME_1':'Bueng Kan','properties.ID_1':79,'geometry.type':'Polygon','geometry.coordinates':[[[104.1914672851562,18.020500183105582],[104.14794921875028,18.01201057434082],[104.11399078369169,17.996362686157227],[104.14251708984375,17.953010559081974],[104.12586212158197,17.9237003326416],[104.07554626464844,17.877380371093807],[104.03219604492182,17.907739639282283],[103.99871063232428,17.88594055175787],[103.98927307128918,17.860471725463867],[104.01936340332037,17.80808830261236],[104.00180053710943,17.78559112548828],[103.96311950683622,17.772718429565373],[103.9441986083985,17.781490325927734],[103.9197616577149,17.81702995300293],[103.92793273925781,17.83284187316906],[103.9074783325197,17.87894058227539],[103.85820007324247,17.881202697753963],[103.7969589233399,17.84066963195812],[103.76860046386713,17.87945938110363],[103.78594970703136,17.92358970642084],[103.75621032714838,17.913850784301815],[103.7203521728515,17.93768119812023],[103.6286010742187,17.98085975646984],[103.60929870605497,18.033781051635856],[103.5629272460938,18.060880661010856],[103.51786041259766,18.073650360107365],[103.4964675903322,18.05924987792963],[103.5065307617187,17.997810363769645],[103.47743225097662,18.003730773925724],[103.44138336181646,17.98147010803217],[103.40622711181658,17.989049911499023],[103.3949584960937,18.021299362182617],[103.36289978027355,18.02691078186035],[103.34513854980497,18.047491073608455],[103.31449127197283,18.055549621582088],[103.29483795166033,18.106170654296818],[103.30110168457043,18.151790618896598],[103.33817291259766,18.145561218261776],[103.35848999023466,18.16118049621582],[103.32775115966803,18.202249526977596],[103.28376007080072,18.232488632202205],[103.26888275146479,18.257921218872127],[103.2955932617187,18.271270751953182],[103.30132293701189,18.302431106567496],[103.27942657470703,18.335149765014762],[103.25740814209013,18.33401107788086],[103.24874877929693,18.366689682006836],[103.31510162353516,18.43177986145031],[103.35025024414068,18.431850433349553],[103.41491699218756,18.4461612701416],[103.45947265625,18.42271995544445],[103.5213470458985,18.42387962341303],[103.57594299316418,18.40230941772461],[103.61933135986328,18.39822959899908],[103.70005035400413,18.340360641479492],[103.81086730957054,18.338340759277344],[103.83436584472685,18.327220916748104],[103.85343170166044,18.285520553588924],[103.88066101074224,18.28607940673828],[103.92568969726585,18.325260162353516],[103.97753906250006,18.332412719726562],[104.06485748291033,18.214580535888786],[104.11801910400402,18.102020263671875],[104.1914672851562,18.020500183105582]]]}
B1 = pd.DataFrame([B1])
features = pd.concat([features, B1], ignore_index=True)
features['geometry.coordinates']

In [ ]:
# สร้างพจนานุกรมสำหรับจังหวัดในประเทศไทย
province_dict = {
    'กระบี่': 'Krabi',
    'กรุงเทพมหานคร': 'Bangkok Metropolis',
    'กาญจนบุรี': 'Kanchanaburi',
    'กาฬสินธุ์': 'Kalasin',
    'กำแพงเพชร': 'Kamphaeng Phet',
    'ขอนแก่น': 'Khon Kaen',
    'จันทบุรี': 'Chanthaburi',
    'ฉะเชิงเทรา': 'Chachoengsao',
    'ชลบุรี': 'Chon Buri',
    'ชัยนาท': 'Chai Nat',
    'ชัยภูมิ': 'Chaiyaphum',
    'ชุมพร': 'Chumphon',
    'เชียงราย': 'Chiang Rai',
    'เชียงใหม่': 'Chiang Mai',
    'ตรัง': 'Trang',
    'ตราด': 'Trat',
    'ตาก': 'Tak',
    'นครนายก': 'Nakhon Nayok',
    'นครปฐม': 'Nakhon Pathom',
    'นครพนม': 'Nakhon Phanom',
    'นครราชสีมา': 'Nakhon Ratchasima',
    'นครศรีธรรมราช': 'Nakhon Si Thammarat',
    'นครสวรรค์': 'Nakhon Sawan',
    'นนทบุรี': 'Nonthaburi',
    'นราธิวาส': 'Narathiwat',
    'น่าน': 'Nan',
    'บึงกาฬ': 'Bueng Kan',
    'บุรีรัมย์': 'Buri Ram',
    'ปทุมธานี': 'Pathum Thani',
    'ประจวบคีรีขันธ์': 'Prachuap Khiri Khan',
    'ปราจีนบุรี': 'Prachin Buri',
    'ปัตตานี': 'Pattani',
    'พระนครศรีอยุธยา': 'Phra Nakhon Si Ayutthaya',
    'พะเยา': 'Phayao',
    'พังงา': 'Phangnga',
    'พัทลุง': 'Phatthalung',
    'พิจิตร': 'Phichit',
    'พิษณุโลก': 'Phitsanulok',
    'เพชรบุรี': 'Phetchaburi',
    'เพชรบูรณ์': 'Phetchabun',
    'แพร่': 'Phrae',
    'ภูเก็ต': 'Phuket',
    'มหาสารคาม': 'Maha Sarakham',
    'มุกดาหาร': 'Mukdahan',
    'แม่ฮ่องสอน': 'Mae Hong Son',
    'ยะลา': 'Yala',
    'ยโสธร': 'Yasothon',
    'ร้อยเอ็ด': 'Roi Et',
    'ระนอง': 'Ranong',
    'ระยอง': 'Rayong',
    'ราชบุรี': 'Ratchaburi',
    'ลพบุรี': 'Lop Buri',
    'ลำปาง': 'Lampang',
    'ลำพูน': 'Lamphun',
    'เลย': 'Loei',
    'ศรีสะเกษ': 'Si Sa Ket',
    'สกลนคร': 'Sakon Nakhon',
    'สงขลา': 'Songkhla',
    'สตูล': 'Satun',
    'สมุทรปราการ': 'Samut Prakan',
    'สมุทรสงคราม': 'Samut Songkhram',
    'สมุทรสาคร': 'Samut Sakhon',
    'สระแก้ว': 'Sa Kaeo',
    'สระบุรี': 'Saraburi',
    'สิงห์บุรี': 'Sing Buri',
    'สุโขทัย': 'Sukhothai',
    'สุพรรณบุรี': 'Suphan Buri',
    'สุราษฎร์ธานี': 'Surat Thani',
    'สุรินทร์': 'Surin',
    'หนองคาย': 'Nong Khai',
    'หนองบัวลำภู': 'Nong Bua Lam Phu',
    'อ่างทอง': 'Ang Thong',
    'อุดรธานี': 'Udon Thani',
    'อุทัยธานี': 'Uthai Thani',
    'อุตรดิตถ์': 'Uttaradit',
    'อุบลราชธานี': 'Ubon Ratchathani',
    'อำนาจเจริญ': 'Amnat Charoen'
}


province_df = pd.DataFrame(list(province_dict.items()), columns=['province_thai', 'province_eng'])
pfmerge= pd.merge( province_df, features, left_on='province_eng', right_on='properties.NAME_1' )
pfmerge_2 =  pd.merge( groupmap_df, pfmerge, left_on='BUDGETARY_UNIT', right_on='province_thai' )
pfmerge_2

In [ ]:
pivlot_pfmerge_2 = pfmerge_2.pivot_table(index='BUDGETARY_UNIT',columns='FISCAL_YEAR',values='AMOUNT',aggfunc='sum'  ).reset_index()
pivlot_pfmerge_2= pd.merge( pivlot_pfmerge_2, pfmerge, left_on='BUDGETARY_UNIT', right_on='province_thai' )
pivlot_pfmerge_2
pivlot_pfmerge_2.columns = pivlot_pfmerge_2.columns.astype(str)
pivlot_pfmerge_2

In [ ]:
import geopandas as gpd
import json

# โหลด GeoJSON
with open('thailand-provinces.geojson', encoding='utf-8') as f:
    features = json.load(f)

# แปลง GeoJSON เป็น GeoDataFrame
geopan1 = gpd.GeoDataFrame.from_features(features['features'])



In [ ]:
variable_list = ['2023', '2024', '2025']
alt.Chart(geopan1).mark_geoshape().encode(
    tooltip=[
        alt.Tooltip('NAME_1:N', title='Province'),
        alt.Tooltip(alt.repeat('row'), title='Value',format=",.2f")
    ],
    color=alt.Color(alt.repeat('row'), type='quantitative')  # เปลี่ยนเป็น color=
).transform_lookup(
    lookup='NAME_1',
    from_=alt.LookupData(pivlot_pfmerge_2, 'province_eng', variable_list)
).properties(
    width=500,
    height=300
).project(
    type='mercator'
).repeat(
    row=variable_list
).resolve_scale(
    color='independent'
)


In [ ]:
df66_67_68['BUDGETARY_UNIT'].count()

In [ ]:
map_df2 = df66_67_68.loc[
    (df66_67_68['MINISTRY'] == 'องค์กรปกครองส่วนท้องถิ่น') &
    (~df66_67_68['BUDGETARY_UNIT'].str.contains('เทศบาล')) & (~df66_67_68['BUDGETARY_UNIT'].str.contains('พัทยา')),
    :
]

map_df2['BUDGETARY_UNIT'] = map_df2['BUDGETARY_UNIT'].replace('องค์การบริหารส่วนจังหวัดอุบลฯ','อุบลราชธานี')
map_df2['BUDGETARY_UNIT'] = map_df2['BUDGETARY_UNIT'].replace('องค์การบริการส่วนจังหวัดกระบี่','กระบี่')


map_df2['BUDGETARY_UNIT'] = map_df2['BUDGETARY_UNIT'].str.replace('องค์การบริหารส่วนจังหวัด', '', regex=False)
map_df2['BUDGETARY_UNIT'] = map_df2['BUDGETARY_UNIT'].replace('กระบี','กระบี่')

pfmerge_4 =  pd.merge( map_df2, pfmerge, left_on='BUDGETARY_UNIT', right_on='province_thai' )
pfmerge_4

pivlot_pfmerge_4 = pfmerge_4.pivot_table(index='BUDGETARY_UNIT',columns='FISCAL_YEAR',values='AMOUNT',aggfunc='sum'  ).reset_index()
pivlot_pfmerge_4= pd.merge( pivlot_pfmerge_4, pfmerge, left_on='BUDGETARY_UNIT', right_on='province_thai' )
pivlot_pfmerge_4
pivlot_pfmerge_4.columns = pivlot_pfmerge_4.columns.astype(str)
pivlot_pfmerge_4_local_withB = pivlot_pfmerge_4
pivlot_pfmerge_4_local_withoutB  = pivlot_pfmerge_4[pivlot_pfmerge_4['province_thai'] != 'กรุงเทพมหานคร']
pivlot_pfmerge_4_local_withonlyB = pivlot_pfmerge_4[pivlot_pfmerge_4['province_thai'] == 'กรุงเทพมหานคร']
pivlot_pfmerge_4['BUDGETARY_UNIT'].unique()

In [ ]:

# สร้างกราฟสำหรับมีกรุงเทพฯ
variable_list = ['2023', '2024', '2025']
local_with_bangkok = alt.Chart(geopan1).mark_geoshape().encode(
    tooltip=[
        alt.Tooltip('NAME_1:N', title='Province'),
        alt.Tooltip(alt.repeat('row'), title='Value',format=",.2f")
    ],
    color=alt.Color(alt.repeat('row'), type='quantitative')  # เปลี่ยนเป็น color=
).transform_lookup(
    lookup='NAME_1',
    from_=alt.LookupData(pivlot_pfmerge_4_local_withB, 'province_eng', variable_list)
).properties(
    width=500,
    height=300
).project(
    type='mercator'
).repeat(
    row=variable_list
).resolve_scale(
    color='independent'
).properties(
    title='Local and Bangkok '
).configure_title(
    fontSize=20,  # เปลี่ยนขนาดตัวอักษร
    anchor='middle'  # จัดตำแหน่งชื่อให้อยู่กลาง
)

local_with_bangkok


In [ ]:
local_without_bangkok = alt.Chart(geopan1).mark_geoshape().encode(
    tooltip=[
        alt.Tooltip('NAME_1:N', title='Province'),
        alt.Tooltip(alt.repeat('row'), title='Value',format=",.2f")
    ],
    color=alt.Color(alt.repeat('row'), type='quantitative')  # เปลี่ยนเป็น color=
).transform_lookup(
    lookup='NAME_1',
    from_=alt.LookupData(pivlot_pfmerge_4_local_withoutB, 'province_eng', variable_list)
).properties(
    width=500,
    height=300
).project(
    type='mercator'
).repeat(
    row=variable_list
).resolve_scale(
    color='independent'
).properties(
    title='Only Local '
).configure_title(
    fontSize=20,  # เปลี่ยนขนาดตัวอักษร
    anchor='middle'  # จัดตำแหน่งชื่อให้อยู่กลาง
)
local_without_bangkok

In [ ]:
local_only_bangkok = alt.Chart(geopan1).mark_geoshape().encode(
    tooltip=[
        alt.Tooltip('NAME_1:N', title='Province'),
        alt.Tooltip(alt.repeat('row'), title='Value',format=",.2f")
    ],
    color=alt.Color(alt.repeat('row'), type='quantitative')  # เปลี่ยนเป็น color=
).transform_lookup(
    lookup='NAME_1',
    from_=alt.LookupData(pivlot_pfmerge_4_local_withonlyB, 'province_eng', variable_list)
).properties(
    width=500,
    height=300
).project(
    type='mercator'
).repeat(
    row=variable_list
).resolve_scale(
    color='independent'
).properties(
    title='Only Bangkok '
).configure_title(
    fontSize=20,  # เปลี่ยนขนาดตัวอักษร
    anchor='middle'  # จัดตำแหน่งชื่อให้อยู่กลาง
)
local_only_bangkok

In [ ]:
provincedata = pivlot_pfmerge_4_local_withB.loc[:,['BUDGETARY_UNIT','2023','2024','2025']]
provincedata['2023_rank']= provincedata['2023'].rank(ascending=False)
provincedata['2024_rank']= provincedata['2024'].rank(ascending=False)
provincedata['2025_rank']= provincedata['2025'].rank(ascending=False)
provincedata['2023_rank']= provincedata['2023_rank'].astype(int)
provincedata['2024_rank']= provincedata['2024_rank'].astype(int)
provincedata['2025_rank']= provincedata['2025_rank'].astype(int)
provincedata = provincedata.sort_values(by=['2023_rank','2024_rank','2025_rank'])
provincedata



In [ ]:
pivlot_pfmerge_2
pivlot_pfmerge_2rank = pivlot_pfmerge_2.loc[:,['BUDGETARY_UNIT','2023','2024','2025']]


pivlot_pfmerge_2rank['2023_rank']= pivlot_pfmerge_2rank['2023'].rank(ascending=False)
pivlot_pfmerge_2rank['2024_rank']= pivlot_pfmerge_2rank['2024'].rank(ascending=False)
pivlot_pfmerge_2rank['2025_rank']= pivlot_pfmerge_2rank['2025'].rank(ascending=False)
pivlot_pfmerge_2rank['2023_rank']= pivlot_pfmerge_2rank['2023_rank'].astype(int)
pivlot_pfmerge_2rank['2024_rank']= pivlot_pfmerge_2rank['2024_rank'].astype(int)
pivlot_pfmerge_2rank['2025_rank']= pivlot_pfmerge_2rank['2025_rank'].astype(int)
pivlot_pfmerge_2rank['2023_2024_change'] = (pivlot_pfmerge_2rank['2024']-pivlot_pfmerge_2rank['2023'])/pivlot_pfmerge_2rank['2023']*100
pivlot_pfmerge_2rank['2024_2025_change'] = (pivlot_pfmerge_2rank['2025']-pivlot_pfmerge_2rank['2024'])/pivlot_pfmerge_2rank['2024']*100


#pivlot_pfmerge_2rank = pivlot_pfmerge_2rank.loc[:,['BUDGETARY_UNIT','2023_rank','2024_rank','2025_rank']]
pivlot_pfmerge_2rank['Mean_of_Absolute_change']= (abs(pivlot_pfmerge_2rank['2024_rank']-pivlot_pfmerge_2rank['2023_rank'])+abs(pivlot_pfmerge_2rank['2025_rank']-pivlot_pfmerge_2rank['2024_rank']))/2
pivlot_pfmerge_2rank.sort_values(by=['2023_rank'])

In [ ]:
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานบูรณาการบริหารจัดการทรัพยากรน้ำ บาท','แผนงานบูรณาการบริหารจัดการทรัพยากรน้ำ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานบูรณาการป้องกัน ปราบปราม และแก้ไขปัญหายาเสพติด','แผนงานบูรณาการป้องกัน ปราบปราม และบำบัดรักษาผู้ติดยาเสพติด')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานบูรณาการป้องกันปราบปรามและบำบัดรักษาผู้ติดยาเสพติด','แผนงานบูรณาการป้องกัน ปราบปราม และบำบัดรักษาผู้ติดยาเสพติด')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานบูรณาการพัฒนาและส่งเสริมเศรษฐกิจฐานราก','แผนงานยุทธศาสตร์พัฒนาและส่งเสริมเศรษฐกิจฐานราก')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์การส่งเสริมวิสาหกิจขนาดกลางและขนาดย่อม ที่เข้มแข็ง แข่งขันได้','แผนงานยุทธศาสตร์การส่งเสริมวิสาหกิจขนาดกลางและขนาดย่อมที่เข้มแข็ง แข่งขันได้')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์การส่งเสริมวิสาหกิจขนาดกลางและขนาดย่อมที่เข้มแข็ง','แผนงานยุทธศาสตร์การส่งเสริมวิสาหกิจขนาดกลางและขนาดย่อมที่เข้มแข็ง แข่งขันได้')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์การส่งเสริมวิสาหกิจขนาดกลางและขนาดย่อมที่เข้มแข็งแข่งขันได้','แผนงานยุทธศาสตร์การส่งเสริมวิสาหกิจขนาดกลางและขนาดย่อมที่เข้มแข็ง แข่งขันได้')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์ปรับเปลี่ยนค่านิยมและวัฒนธรรม 138,799,900','แผนงานยุทธศาสตร์ปรับเปลี่ยนค่านิยมและวัฒนธรรม')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์พัฒนาศักยภาพการป้องกันประเทศ','แผนงานยุทธศาสตร์พัฒนาศักยภาพการป้องกันประเทศ และความพร้อมเผชิญภัยคุกคามทุกมิติ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์พัฒนาและเสริมสร้างการเมืองในระบอบประชาธิปไตย','แผนงานยุทธศาสตร์พัฒนาและเสริมสร้างการเมืองในระบอบประชาธิปไตยอันมีพระมหากษัตริย์ทรงเป็นประมุข')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์พัฒนาและเสริมสร้างการเมืองในระบอบประชาธิปไตย อันมีพระมหากษัตริย์ทรงเป็นประมุข','แผนงานยุทธศาสตร์พัฒนาและเสริมสร้างการเมืองในระบอบประชาธิปไตยอันมีพระมหากษัตริย์ทรงเป็นประมุข')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์พัฒนาและเสริมสร้างการเมืองในระบอบประชาธิปไตยอันมีพระมหากษัตริย์','แผนงานยุทธศาสตร์พัฒนาและเสริมสร้างการเมืองในระบอบประชาธิปไตยอันมีพระมหากษัตริย์ทรงเป็นประมุข')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์บริหารจัดการทรัพยากรน้ำ','แผนงานบูรณาการบริหารจัดการทรัพยากรน้ำ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์เสริมสร้างประสิทธิภาพการบริหารจัดการทรัพยากรน้ำ','แผนงานบูรณาการบริหารจัดการทรัพยากรน้ำ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์พัฒนาด้านคมนาคมและระบบโลจิสติกส์','แผนงานบูรณาการพัฒนาด้านคมนาคมและระบบโลจิสติกส์')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์ยกระดับกระบวนทัศน์เพื่อกำหนดอนาคตประเทศด้านทรัพยากรธรรมชาติ และสิ่งแวดล้อม','แผนงานยุทธศาสตร์ยกระดับกระบวนทัศน์เพื่อกำหนดอนาคตประเทศด้านทรัพยากรธรรมชาติ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์ยกระดับกระบวนทัศน์เพื่อกำหนดอนาคตประเทศด้านทรัพยากรธรรมชาติและสิ่งแวดล้อม','แผนงานยุทธศาสตร์ยกระดับกระบวนทัศน์เพื่อกำหนดอนาคตประเทศด้านทรัพยากรธรรมชาติ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์สร้างการเติบโตอย่างยั่งยืน อนุรักษ์ ฟื้นฟู และป้องกัน การทำลายทรัพยากรธรรมชาติ','แผนงานยุทธศาสตร์สร้างการเติบโตอย่างยั่งยืน อนุรักษ์ ฟื้นฟู และป้องกันการทำลายทรัพยากรธรรมชาติ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์สร้างการเติบโตอย่างยั่งยืน อนุรักษ์ ฟื้นฟู และป้องกันการทำลาย ทรัพยากรธรรมชาติ','แผนงานยุทธศาสตร์สร้างการเติบโตอย่างยั่งยืน อนุรักษ์ ฟื้นฟู และป้องกันการทำลายทรัพยากรธรรมชาติ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์สร้างการเติบโตอย่างยั่งยืน อนุรักษ์ ฟื้นฟู และป้องกันการทำลายทรัพยากรธรรมชาติ','แผนงานยุทธศาสตร์สร้างการเติบโตอย่างยั่งยืน อนุรักษ์ ฟื้นฟู และป้องกันการทำลายทรัพยากรธรรมชาติ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์สร้างการเติบโตอย่างยั่งยืนอนุรักษ์ฟื้นฟูและป้องกันการทำลายทรัพยากรธรรมชาติ','แผนงานยุทธศาสตร์สร้างการเติบโตอย่างยั่งยืน อนุรักษ์ ฟื้นฟู และป้องกันการทำลายทรัพยากรธรรมชาติ')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์เพื่อสนับสนุนด้านการสร้างการเติบโตบนคุณภาพชีวิตที่เป็นมิตร ต่อสิ่งแวดล้อม','แผนงานยุทธศาสตร์เพื่อสนับสนุนด้านการสร้างการเติบโตบนคุณภาพชีวิตที่เป็นมิตรต่อสิ่งแวดล้อม')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานยุทธศาสตร์เพื่อสนับสนุนด้านการสร้างโอกาส และความเสมอภาคทางสังคม','แผนงานยุทธศาสตร์เพื่อสนับสนุนด้านการสร้างโอกาสและความเสมอภาคทางสังคม')
df66_67_68['BUDGET_PLAN'] = df66_67_68['BUDGET_PLAN'].replace('แผนงานพื้นฐานด้านการปรับสมดุลและพัฒนาระบบการบริหารจัดการภาครัฐ','แผนงานยุทธศาสตร์เพื่อสนับสนุนด้านการปรับสมดุลและพัฒนาระบบการบริหารจัดการภาครัฐ')

df66_67_68['BUDGET_PLAN'].unique()
plan_df = df66_67_68.pivot_table(index='BUDGET_PLAN',
                                  columns='FISCAL_YEAR',
                                  values='AMOUNT',
                                  aggfunc='sum').reset_index()
plan_df['BUDGET_PLAN'].unique()

In [ ]:
plan_df.sort_values(by=2023,ascending=False)

In [ ]:
# จัดเรียง DataFrame ตามปี 2023
sorted_plan_df = plan_df.sort_values(by=2023, ascending=False)

# เก็บแค่ 5 แถวแรก
top_5 = sorted_plan_df.head(9)

# รวมแถวที่เหลือเป็น "Other"
other_row = sorted_plan_df.iloc[5:].sum(numeric_only=True).to_frame().T
other_row['BUDGET_PLAN'] = 'Other'  # เปลี่ยนชื่อเป็น 'Other'

# รวม DataFrame ทั้งสอง
result_df = pd.concat([top_5, other_row], ignore_index=True)
result_df.columns = result_df.columns.astype(str)

# แสดงผล
result_df


In [ ]:
#

In [ ]:
bar_chart1 = alt.Chart(result_df).mark_bar().encode(
    x=alt.X('BUDGET_PLAN:O', title='Budget Plan',sort=alt.EncodingSortField(field='2023', order='descending')),
    y=alt.Y('2023:Q', title='Amount'),
    color=alt.Color('BUDGET_PLAN:N', title='Fiscal Year'),
     tooltip=[
        alt.Tooltip('BUDGET_PLAN:O', title='Budget Plan'),
        alt.Tooltip('2023:Q', title='Amount', format=',')]
).properties(
    width=800,
    height=400,
         title={
        "text": ["2023"],  # ข้อความ Title
        "fontSize": 16,  # ขนาดตัวอักษร
        "anchor": "middle",  # จัดให้อยู่กลาง
        "fontWeight": "bold"  # หนักตัวอักษร
    }
)
bar_chart2 = alt.Chart(result_df).mark_bar().encode(
    x=alt.X('BUDGET_PLAN:O', title='Budget Plan',sort=alt.EncodingSortField(field='2024', order='descending')),
    y=alt.Y('2024:Q', title='Amount'),
    color=alt.Color('BUDGET_PLAN:N', title='Fiscal Year'),
     tooltip=[
        alt.Tooltip('BUDGET_PLAN:O', title='Budget Plan'),
        alt.Tooltip('2024:Q', title='Amount', format=',')]
).properties(
    width=800,
    height=400,
         title={
        "text": ["2024"],  # ข้อความ Title
        "fontSize": 16,  # ขนาดตัวอักษร
        "anchor": "middle",  # จัดให้อยู่กลาง
        "fontWeight": "bold"  # หนักตัวอักษร
    }
)
bar_chart3 = alt.Chart(result_df).mark_bar().encode(
    x=alt.X('BUDGET_PLAN:O', title='Budget Plan',sort=alt.EncodingSortField(field='2025', order='descending')),
    y=alt.Y('2025:Q', title='Amount'),
    color=alt.Color('BUDGET_PLAN:N', title='Fiscal Year'),
     tooltip=[
        alt.Tooltip('BUDGET_PLAN:O', title='Budget Plan'),
        alt.Tooltip('2025:Q', title='Amount', format=',')]
).properties(
    width=800,
    height=400,
     title={
        "text": ["2025"],  # ข้อความ Title
        "fontSize": 16,  # ขนาดตัวอักษร
        "anchor": "middle",  # จัดให้อยู่กลาง
        "fontWeight": "bold"  # หนักตัวอักษร
    }
)
bar_chart1|bar_chart2|bar_chart3

In [ ]:
local1 = map_df2[map_df2['FISCAL_YEAR'] == 2024]

local1


In [ ]:
local1PI = local1.pivot_table(index='ITEM_DESCRIPTION', columns='FISCAL_YEAR', values='AMOUNT', aggfunc='sum').reset_index()
local1PI.columns = local1PI.columns.astype(str)

local1PI

In [ ]:
import pandas as pd
import altair as alt

# สมมติว่า local1 เป็น DataFrame ของคุณ
# เลือกข้อมูลที่มีจำนวนสูงสุด 5 อันดับแรก
top5 = local1PI.nlargest(10, '2024')

# สร้างกราฟ bar chart ด้วย Altair
chart = alt.Chart(top5).mark_bar().encode(
    x=alt.X('ITEM_DESCRIPTION:O', sort=alt.SortField('AMOUNT', order='descending')),
    y='2024',
    color='ITEM_DESCRIPTION',  # สีตาม ITEM_DESCRIPTION
    tooltip=[
        alt.Tooltip('ITEM_DESCRIPTION', title='Item Description'),
        alt.Tooltip('2024:Q', title='Total Amount', format=',')  # ใช้ format=',' เพื่อใส่ลูกน้ำ
    ]
).properties(
    width=800,  # กำหนดความกว้าง
    height=400,  # กำหนดความสูง
    title=alt.TitleParams(
        text='ปี 2024 องค์กรปกครองส่วนท้องถิ่นใช้เงินไปกับอะไรไปมากที่สุด 10 อันดับแรก',  # กำหนดชื่อกราฟ
        anchor='middle',  # จัดตำแหน่งชื่อกราฟให้อยู่ตรงกลาง
        fontSize=20,  # ปรับขนาดตัวอักษร
        fontWeight='bold'  # ทำตัวอักษรให้หนา
    )
)



chart


In [ ]:
cenb = df66_67_68[df66_67_68['BUDGETARY_UNIT'] == 'งบกลาง']
cenb['ITEM_DESCRIPTION'] = cenb['ITEM_DESCRIPTION'].replace('เงินสำรองจ่ายเพื่อกรณีฉุกเฉินหรือจ าเป็น','เงินสำรองจ่ายเพื่อกรณีฉุกเฉินหรือจำเป็น')
cenb['ITEM_DESCRIPTION'] = cenb['ITEM_DESCRIPTION'].replace('เงินเดือนเงินเดือนและเงินปรับวุฒิข้าราชการ','เงินเลื่อนเงินเดือนและเงินปรับวุฒิข้าราชการ')


cenb

In [ ]:
cenb_pi = cenb.pivot_table(index='ITEM_DESCRIPTION', columns='FISCAL_YEAR', values='AMOUNT', aggfunc='sum').reset_index()
cenb_pi.columns = cenb_pi.columns.astype(str)

In [ ]:
cenb_pi.fillna(0, inplace=True)
cenb_pi
unpivoted_cenb_pi = cenb_pi.melt(id_vars=['ITEM_DESCRIPTION'],
                    var_name='Year',
                    value_name='Amount')
unpivoted_cenb_pi

In [ ]:
import altair as alt

import altair as alt

alt.Chart(unpivoted_cenb_pi).mark_bar().encode(
    x=alt.X("ITEM_DESCRIPTION:N", title="Item Description"),
    y=alt.Y("Amount:Q", title="Amount"),
    color="Year:N",
    column="Year:N"  # ใช้สำหรับการแบ่งกลุ่ม
).properties(
    width=150,
    height=300
)




In [ ]:
unc = alt.Chart(unpivoted_cenb_pi).mark_bar().encode(
    x=alt.X('Year:N', title='Category'),
    y=alt.Y('Amount:Q', title='Amount'),
    color='ITEM_DESCRIPTION:N',
    column='ITEM_DESCRIPTION:N'  # ใช้ column เพื่อจัดกลุ่มตาม Group
).properties(
    width=500,
    height=400
)


unc

In [ ]:
import altair as alt

# สร้างกราฟแท่ง
chart = alt.Chart(unpivoted_cenb_pi).mark_bar().encode(
    x=alt.X('sum(Amount):Q', title='Total Amount', stack='normalize'),  # รวมจำนวนเงิน
    y=alt.Y('ITEM_DESCRIPTION:N', title='Item Description'),
    color='Year:N',  # ใช้สีตามปี
    tooltip=[alt.Tooltip('Year:N', title='Year'), alt.Tooltip('sum(Amount):Q', title='Total Amount'), format=',']
).properties(
    width=600,
    height=400
)

chart



In [ ]:
import altair as alt

# สร้างกราฟแท่งพร้อม tooltip ที่มีลูกน้ำ
chart = alt.Chart(unpivoted_cenb_pi).mark_bar().encode(
    x=alt.X('sum(Amount):Q', title='Total Amount', stack='normalize'),  # รวมจำนวนเงิน
    y=alt.Y('ITEM_DESCRIPTION:N', title='Item Description'),
    color='Year:N',  # ใช้สีตามปี
    tooltip=[
        alt.Tooltip('Year:N', title='Year'),
        alt.Tooltip('sum(Amount):Q', title='Total Amount', format=',')  # เพิ่มลูกน้ำ
    ]
).properties(
    width=1000,
    height=800,
        title=alt.TitleParams(
        text='งบกลางในแต่ละปีถูกใช้ไปกับอะไรบ้าง',
        fontSize=20,  # ขนาดตัวอักษร
        anchor='middle'  # ตำแหน่งอยู่กลาง
    )
)

chart


In [ ]:
chart = alt.Chart(unpivoted_cenb_pi).mark_bar().encode(
    x=alt.X('sum(Amount):Q', title='Total Amount', stack='normalize'),  # รวมจำนวนเงิน
    y=alt.Y('ITEM_DESCRIPTION:N', title='Item Description', sort='-x'),  # เพิ่ม sort ถ้าจำเป็น
    color='Year:N',  # ใช้สีตามปี
    tooltip=[
        alt.Tooltip('Year:N', title='Year'),
        alt.Tooltip('sum(Amount):Q', title='Total Amount', format=',')  # เพิ่มลูกน้ำ
    ]
).properties(
    width=1000,
    height=1000,  # ปรับความสูงกราฟ
    title=alt.TitleParams(
        text='งบกลางในแต่ละปีถูกใช้ไปกับอะไรบ้าง',
        fontSize=20,  # ขนาดตัวอักษร
        anchor='middle'  # ตำแหน่งอยู่กลาง
    )
).configure_axisY(
    labelAngle=-45  # หมุนป้ายชื่อในแกน Y
)
chart

In [ ]:
gdp_tax = pd.read_excel('Budget_Gdp_taxreva.xlsx')
gdp_tax = gdp_tax.dropna()
gdp_tax['ปี'] = gdp_tax['ปี']+543
gdp_tax

In [ ]:
unpivoted_gdp_tax = pd.melt(gdp_tax, id_vars=['ปี'], value_name='value')
unpivoted_gdp_tax

In [ ]:
A = alt.Chart(gdp_tax).mark_line().encode(
    y='ดุล:Q',
    x='ปี:N'
)
A

In [ ]:
correlation_matrix = gdp_tax.corr()

correlation_matrix

In [ ]:
styled_heatmap = correlation_matrix.style.background_gradient(cmap='coolwarm').format("{:.2f}").set_properties(**{'width': '75px'}).set_table_styles([{'selector': 'tr', 'props': [('height', '75px')]}])

# แสดง hea

# แสดง heatmap
styled_heatmap


In [ ]:
AA = pd.read_excel('Budget_Gdp_taxreva.xlsx')
AA = AA.dropna()
AA['ปี'] = AA['ปี']+543


In [ ]:
AA['growth rate งบประมาณ'] = AA['growth rate งบประมาณ']*100
AA['growth รายได้รัฐบาล'] = AA['growth รายได้รัฐบาล']*100
AA['ดุล growth']  = AA['ดุล growth']*100
AA

In [ ]:
# ปรับ line chart
line01 = alt.Chart(AA).mark_line().encode(
    x=alt.X('ปี:N', axis=alt.Axis(labelFontSize=12, titleFontSize=14)),
    y=alt.Y('growth งบประมาณ', axis=alt.Axis(labelFontSize=12, titleFontSize=14))
).properties(
    width=500, height=300  # ปรับขนาดกราฟ
)

line02 = alt.Chart(AA).mark_line().encode(
    x=alt.X('ปี:N', axis=alt.Axis(labelFontSize=12, titleFontSize=14)),
    y=alt.Y('gdp growth', axis=alt.Axis(labelFontSize=12, titleFontSize=14))
).properties(
    width=500, height=300
)

line03 = alt.Chart(AA).mark_line().encode(
    x=alt.X('ปี:N', axis=alt.Axis(labelFontSize=12, titleFontSize=14)),
    y=alt.Y('ดุล', axis=alt.Axis(labelFontSize=12, titleFontSize=14))
).properties(
    width=500, height=300
)

line04 = alt.Chart(AA).mark_line().encode(
    x=alt.X('ปี:N', axis=alt.Axis(labelFontSize=12, titleFontSize=14)),
    y=alt.Y('รายได้รัฐบาล (ล้านบาท)', axis=alt.Axis(labelFontSize=12, titleFontSize=14))
).properties(
    width=500, height=300
)

# รวมกราฟเข้าด้วยกัน
(line01 & line04) | (line02 & line03)


In [ ]:
modeltrain = AA
Y = AA['งบประมาณ']
modeltrain.drop('งบประมาณ', axis=1, inplace=True)
modeltrain.drop('growth งบประมาณ', axis=1, inplace=True)
modeltrain.drop('growth rate งบประมาณ', axis=1, inplace=True)
modeltrain.drop('gdp growth', axis=1, inplace=True)
modeltrain.drop('growth รายได้รัฐบาล', axis=1, inplace=True)
modeltrain.drop('ดุล growth', axis=1, inplace=True)
modeltrain

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso


# 1. สร้าง DataFrame หรือโหลดข้อมูล
# สมมติว่า df คือ DataFrame ที่มีข้อมูลของคุณ
# แยก features (X) และ target (y)
X = modeltrain  # ใส่ชื่อคอลัมน์ที่เป็น features ของคุณ
y = Y   # ใส่ชื่อคอลัมน์ที่เป็น target

# 2. แบ่งข้อมูลเป็นชุด train และ test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. สร้างโมเดลและฝึกสอน
alpha = 0.5  # ปรับค่า alpha ตามความเหมาะสม
model = Lasso(alpha=alpha)
model.fit(X_train, y_train)

# 4. ทำนายค่า
y_pred = model.predict(X_test)

# 5. ประเมินผลโมเดล
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


In [ ]:
new_data = pd.DataFrame({
    'ปี': [2569],  # แทนที่ value1 ด้วยค่าที่ต้องการทำนาย
    'รายได้รัฐบาล (ล้านบาท)': [3000000],  # แทนที่ value2 ด้วยค่าที่ต้องการทำนาย
    'ดุล': [-1000000]   # แทนที่ value3 ด้วยค่าที่ต้องการทำนาย
})
predictions = model.predict(new_data)
predictions
